# Deep Hedging pipeline

## Evironment setup

Setup the general environment

In [1]:
import os, sys
from pathlib import Path

main_folder = str(Path.cwd().parent)
sys.path.append(main_folder)

import warnings
warnings.filterwarnings("ignore")

## Market simulator

Simulation of the variable annuity features and market environment

In [2]:
from src.utils import *
from src.features.market_simulator import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
market_model = market_simulator_class(config_file)
market_model.simulation()
market_model.delta_features()
market_model.delta_hedging()

-- Market simulation starts --
--Progress of market simulation: 100.00%
--Storing market features--

-- Hedging instruments array stored --

-- Gains and losses array stored --

-- Deltas input array stored --

-- Cash flow array stored --

-- Input array stored --

--Simulation of simulation features completed--
-- Delta hedging computation starts --
-- Progress of features computation: 
 Shap:8-Simulation:1000-Time:239
--Storing delta hedging features--
--Delta hedging features computation completed--
-- Positions computation starts --

--Positions computation completed and stored--


## Deep Hedging 

Deep RL agent training and hedging strategy generation

**Remark 1**: This segment of the pipeline assumes that the market environment and the CVaR network have already been simulated and trained, respectively. See notebook *cvarnet_pipeline.ipynb* for for more details.

**Remark 2**: This training example illustrates the performance of the RL agent optimized under the assumption fund without local risk penalization. The corresponding results are reported in Table 3 of the paper.


In [3]:
from src.utils import *
from src.models.deep_rl_training import rl_agent

config_file = load_config(os.path.join(main_folder,'cfgs','config_agent.yml'))
config_file_agent = config_file["agent"]
config_file = load_config(os.path.join(main_folder,'cfgs','config_market_simulation.yml'))
config_file_simulation = config_file["simulation"]

_ = rl_agent(config_file_simulation,config_file_agent)

-------------------------------------------------------------
Deep_agent_FFNN_dropout_50_contribution_assupmtion_240_
-------------------------------------------------------------
-----------------------Training start------------------------
Epoch 1, Time elapsed: 0:10:09.303139  - Model saved
  Train - SMSE: 1446.450684, Time: 203.031113, Rate: 715.917297, Mortality: 717.076782, Local-CVaR: 0.000006
  Valid - SMSE: 27.564274, Time: 4.069118, Rate: 11.823818, Mortality: 15.445654, Local-CVaR: 0.000006
Epoch 2, Time elapsed: 0:16:19.605127  - Model saved
  Train - SMSE: 4.116337, Time: 0.575670, Rate: 1.969488, Mortality: 1.953261, Local-CVaR: 0.044081
  Valid - SMSE: 0.029771, Time: 0.004647, Rate: 0.013719, Mortality: 0.019578, Local-CVaR: 0.044081
Epoch 3, Time elapsed: 0:22:58.319568  - Model saved
  Train - SMSE: 0.008848, Time: 0.002057, Rate: 0.006450, Mortality: 0.007862, Local-CVaR: 0.046261
  Valid - SMSE: 0.006661, Time: 0.001777, Rate: 0.004774, Mortality: 0.008009, Local-CV

INFO:tensorflow:Restoring parameters from Deep_agent_FFNN_dropout_50_contribution_assupmtion_240_.ckpt


-------------------------------------------------------------
Deep_agent_FFNN_dropout_50_contribution_assupmtion_240_
-------------------------------------------------------------
---Inference start---
---Inference end---
--- Hedging startegy stored in ../results/Training/.. ---
--- Risk allocation metrics ---
           Expectation  CVaR:95%  Variance
Total         0.069920  0.082425  0.002944
Equity        0.105309 -0.054518  0.000992
Rate         -0.011857  0.061120  0.001004
Mortality    -0.000890  0.053181  0.000948
Time         -0.022643  0.022643  0.000000
